In [8]:
# import 
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
from pandas import DataFrame as df

In [9]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
wiki = soup.find('table')

In [10]:
neigh = wiki.find_all('tr')
d = []
for neigh in neigh:
    cols=neigh.find_all('td')
    cols=[x.text.strip() for x in cols]
    d.append(cols)


In [11]:
columns=['PostalCode','Borough','Neighbourhood']
df_TO = pd.DataFrame(data=d,columns=columns)[1:]
print(df_TO.count())
df_TO.head()

PostalCode       288
Borough          288
Neighbourhood    288
dtype: int64


,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [12]:
#Ignore cells with a borough that is Not assigned.
df_TO_clean =df_TO[df_TO['Borough'] !='Not assigned']

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
for index, row in df_TO_clean.iterrows():
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood']=row['Borough']


In [13]:
df_TO_clean_gr = df_TO_clean.groupby(['PostalCode','Borough'], sort=False).agg( ','.join)
df_TO_clean_gr = df_TO_clean_gr.reset_index()
df_TO_clean_gr.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [14]:
df_TO_clean_gr.shape
df_TO_clean_gr['PostalCode'].head()

0    M3A
1    M4A
2    M5A
3    M6A
4    M7A
Name: PostalCode, dtype: object

In [157]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [15]:
import geocoder

In [22]:
# convert postal code to list to iterate
pc_list = df_TO_clean_gr['PostalCode'].tolist()

In [25]:
# create empty lists to store values
lat_vals=[]
lon_vals=[]

for pc in pc_list:
    lat_lng_coords=None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
        lat_lng_coords = g.latlng

    lat_vals.append(lat_lng_coords[0])
    lon_vals.append(lat_lng_coords[1])
    
df_TO_clean_gr['Latitude'] = lat_vals
df_TO_clean_gr['Longitude'] = lon_vals

df_TO_clean_gr.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752440,-79.329271
1,M4A,North York,Victoria Village,43.730421,-79.313320
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.655120,-79.362640
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723125,-79.451589
4,M7A,Queen's Park,Queen's Park,43.661102,-79.391035
